# AI-Powered Professional Resume Generator

This notebook creates professional resumes using user input and AI enhancement.

## Features:
- Interactive web form for data collection
- Llama model integration for content enhancement
- Professional PDF generation
- AWS S3 upload capability
- Downloadable project structure

## 1. Install Required Dependencies

In [1]:
!pip install transformers torch accelerate reportlab boto3 flask flask-cors pyngrok --quiet
print("✓ All dependencies installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00
✓ All dependencies installed successfully!


## 2. Import Libraries

In [2]:
import json
import os
import io
import zipfile
from datetime import datetime
from pathlib import Path
import base64

# ML/AI Libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# PDF Generation
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY
from reportlab.pdfgen import canvas

# AWS
import boto3
from botocore.exceptions import ClientError

# Web Server
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import threading

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


## 3. Load Llama Model from Hugging Face

In [3]:
class LlamaResumeEnhancer:
  def load_model():
    global model, tokenizer

    # hf_token = "<provide your hf_token>"
    if not hf_token:
        print("⚠️  No HuggingFace token - AI disabled")
        return False

    try:
        print("🤖 Loading Llama model...")
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Llama-2-7b-chat-hf",
            token=hf_token
        )
        model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Llama-2-7b-chat-hf",
            token=hf_token,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True
        )
        print("✅ Llama model loaded!\n")
        return True
    except Exception as e:
        print(f"⚠️  Llama load failed: {str(e)}\n")
        return False

# Initialize the model
print("Initializing Llama model...")
llama_enhancer = LlamaResumeEnhancer()
print("✓ Model ready!")

Initializing Llama model...
✓ Model ready!


## 4. PDF Resume Generator

In [10]:
class ProfessionalResumeGenerator:
    def __init__(self):
        self.styles = getSampleStyleSheet()
        self._create_custom_styles()

    def _create_custom_styles(self):
        """Create custom paragraph styles for the resume."""
        # Title style
        self.styles.add(ParagraphStyle(
            name='CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=24,
            textColor=colors.HexColor('#1a237e'),
            spaceAfter=6,
            alignment=TA_CENTER,
            fontName='Helvetica-Bold'
        ))

        # Contact info style
        self.styles.add(ParagraphStyle(
            name='ContactInfo',
            parent=self.styles['Normal'],
            fontSize=10,
            textColor=colors.HexColor('#424242'),
            alignment=TA_CENTER,
            spaceAfter=12
        ))

        # Section header style
        self.styles.add(ParagraphStyle(
            name='SectionHeader',
            parent=self.styles['Heading2'],
            fontSize=14,
            textColor=colors.HexColor('#1a237e'),
            spaceAfter=6,
            spaceBefore=12,
            fontName='Helvetica-Bold',
            borderColor=colors.HexColor('#1a237e'),
            borderWidth=1,
            borderPadding=3
        ))

        # Body text style
        self.styles.add(ParagraphStyle(
            name='BodyText1',
            parent=self.styles['Normal'],
            fontSize=10,
            alignment=TA_JUSTIFY,
            spaceAfter=6
        ))

        # Bullet points
        self.styles.add(ParagraphStyle(
            name='Bullet1',
            parent=self.styles['Normal'],
            fontSize=10,
            leftIndent=20,
            spaceAfter=3
        ))

    def generate_resume(self, user_data, output_path):
        """Generate a professional PDF resume."""
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )

        story = []

        # Header - Name and Contact
        story.append(Paragraph(user_data.get('name', 'N/A').upper(), self.styles['CustomTitle']))

        # contact_info = f"{user_data.get('mobile', '')} | {user_data.get('email', '')} | {user_data.get('location', '')}"
        # story.append(Paragraph(contact_info, self.styles['ContactInfo']))
        # Contact info with LinkedIn
        contact_parts = [
            user_data.get('mobile', ''),
            user_data.get('email', ''),
            user_data.get('location', '')
        ]

        # Add LinkedIn if provided
        if user_data.get('linkedin'):
            linkedin_url = user_data['linkedin']
            # Create clickable link
            contact_parts.append(f'<a href="{linkedin_url}" color="blue">LinkedIn Profile</a>')

        contact_info = ' | '.join(filter(None, contact_parts))
        story.append(Paragraph(contact_info, self.styles['ContactInfo']))

        story.append(Spacer(1, 12))

        # Professional Summary
        if user_data.get('summary'):
            story.append(Paragraph('PROFESSIONAL SUMMARY', self.styles['SectionHeader']))
            story.append(Paragraph(user_data['summary'], self.styles['BodyText1']))
            story.append(Spacer(1, 12))

        ## Education with CGPA/Percentage
        if user_data.get('qualifications'):
            story.append(Paragraph('EDUCATION', self.styles['SectionHeader']))
            for qual in user_data['qualifications']:
                qual_text = f"<b>{qual.get('degree', '')} {qual.get('specialization', '')}</b> - {qual.get('institution', '')}"

                # Add year and grade on same line
                details = []
                if qual.get('year'):
                    details.append(qual['year'])
                if qual.get('grade'):
                    details.append(f"Grade: {qual['grade']}")

                if details:
                    qual_text += f" ({', '.join(details)})"

                story.append(Paragraph(qual_text, self.styles['BodyText']))
                story.append(Spacer(1, 6))
            story.append(Spacer(1, 6))

        # Professional Experience
        years = user_data.get('years_experience', 0)
        role = user_data.get('current_role', 'N/A')

        story.append(Paragraph('PROFESSIONAL EXPERIENCE', self.styles['SectionHeader']))
        if role.lower() != 'fresher' and years > 0:
            exp_text = f"<b>{role}</b> - {years} years of experience"
            story.append(Paragraph(exp_text, self.styles['BodyText1']))
        else:
            story.append(Paragraph("Fresher - Seeking opportunities to apply skills and grow professionally", self.styles['BodyText']))
        story.append(Spacer(1, 12))

        # Technical Skills
        if user_data.get('skills'):
            story.append(Paragraph('TECHNICAL SKILLS', self.styles['SectionHeader']))
            skills_text = ' • '.join(user_data['skills'])
            story.append(Paragraph(skills_text, self.styles['BodyText1']))
            story.append(Spacer(1, 12))

        # Tech Stack
        if user_data.get('tech_stack'):
            story.append(Paragraph('TECH STACK', self.styles['SectionHeader']))
            tech_text = ' • '.join(user_data['tech_stack'])
            story.append(Paragraph(tech_text, self.styles['BodyText1']))
            story.append(Spacer(1, 12))

        # Projects
        if user_data.get('projects'):
            story.append(Paragraph('PROJECTS', self.styles['SectionHeader']))
            for project in user_data['projects']:
                project_title = f"<b>{project.get('name', 'Untitled Project')}</b>"
                story.append(Paragraph(project_title, self.styles['BodyText1']))

                if project.get('description'):
                    story.append(Paragraph(f"• {project['description']}", self.styles['Bullet']))

                if project.get('technologies'):
                    tech_text = f"<i>Technologies: {project['technologies']}</i>"
                    story.append(Paragraph(tech_text, self.styles['Bullet1']))

                story.append(Spacer(1, 8))

        # Certifications
        if user_data.get('certifications'):
            story.append(Paragraph('CERTIFICATIONS', self.styles['SectionHeader']))
            for cert in user_data['certifications']:
                cert_text = f"• <b>{cert.get('name', '')}</b>"
                if cert.get('issuer'):
                    cert_text += f" - {cert['issuer']}"
                if cert.get('year'):
                    cert_text += f" ({cert['year']})"
                story.append(Paragraph(cert_text, self.styles['Bullet1']))
                story.append(Spacer(1, 4))

        # Build PDF
        doc.build(story)
        print(f"✓ Resume generated: {output_path}")
        return output_path

# Test the generator
resume_generator = ProfessionalResumeGenerator()
print("✓ Resume generator initialized!")

✓ Resume generator initialized!


## 5. AWS S3 Uploader

In [5]:
class S3Uploader:
    def __init__(self, aws_access_key=None, aws_secret_key=None, bucket_name=None, region='us-east-1'):
        """
        Initialize S3 uploader.
        Set credentials as environment variables or pass them here.
        """
        self.aws_access_key = aws_access_key or os.getenv('AWS_ACCESS_KEY_ID')
        self.aws_secret_key = aws_secret_key or os.getenv('AWS_SECRET_ACCESS_KEY')
        self.bucket_name = bucket_name or os.getenv('AWS_BUCKET_NAME')
        self.region = region

        if all([self.aws_access_key, self.aws_secret_key, self.bucket_name]):
            self.s3_client = boto3.client(
                's3',
                aws_access_key_id=self.aws_access_key,
                aws_secret_access_key=self.aws_secret_key,
                region_name=self.region
            )
            print("✓ S3 client initialized")
        else:
            self.s3_client = None
            print("⚠ AWS credentials not configured. S3 upload will be skipped.")

    def upload_file(self, file_path, object_name=None):
        """Upload a file to S3 bucket."""
        if not self.s3_client:
            return None, "AWS S3 not configured"

        if object_name is None:
            object_name = f"resumes/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{os.path.basename(file_path)}"

        try:
            self.s3_client.upload_file(file_path, self.bucket_name, object_name)

            # Generate URL
            url = f"https://{self.bucket_name}.s3.{self.region}.amazonaws.com/{object_name}"
            print(f"✓ File uploaded to S3: {url}")
            return url, None
        except ClientError as e:
            error_msg = f"S3 upload failed: {e}"
            print(f"✗ {error_msg}")
            return None, error_msg

    def configure_credentials(self, aws_access_key, aws_secret_key, bucket_name, region='us-east-1'):
        """Configure AWS credentials dynamically."""
        self.aws_access_key = aws_access_key
        self.aws_secret_key = aws_secret_key
        self.bucket_name = bucket_name
        self.region = region

        self.s3_client = boto3.client(
            's3',
            aws_access_key_id=self.aws_access_key,
            aws_secret_access_key=self.aws_secret_key,
            region_name=self.region
        )
        print("✓ AWS credentials configured")

# Initialize S3 uploader
s3_uploader = S3Uploader()
print("✓ S3 uploader ready")

⚠ AWS credentials not configured. S3 upload will be skipped.
✓ S3 uploader ready


## 6. Flask Web Application

## 7. AWS Credentials Configuration (Optional)

In [8]:
# Configure AWS S3 credentials here (optional)
# You can also set these as environment variables

# Uncomment and fill in your credentials:
# AWS_ACCESS_KEY_ID = "<AWS_ACCESS_KEY_ID>"
# AWS_SECRET_ACCESS_KEY = "<AWS_SECRET_ACCESS_KEY>"
# AWS_BUCKET_NAME = "<AWS_BUCKET_NAME>"
# AWS_REGION = "avp-southeast-2"

s3_uploader.configure_credentials(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_BUCKET_NAME, AWS_REGION)

print("⚠ AWS S3 configuration: Set your credentials above or skip for local-only usage")

✓ AWS credentials configured
⚠ AWS S3 configuration: Set your credentials above or skip for local-only usage


In [6]:
# Create Flask app
app = Flask(__name__)
CORS(app)

# Create output directory
OUTPUT_DIR = Path('/content/resumes')
OUTPUT_DIR.mkdir(exist_ok=True)

@app.route('/')
def home():
    """Serve the frontend HTML."""
    return '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Resume Generator</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        .container {
            max-width: 900px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0, 0, 0, 0.3);
            padding: 40px;
        }

        h1 {
            color: #667eea;
            text-align: center;
            margin-bottom: 10px;
            font-size: 2.5em;
        }

        .subtitle {
            text-align: center;
            color: #666;
            margin-bottom: 30px;
        }

        .form-section {
            margin-bottom: 30px;
            padding: 20px;
            background: #f8f9fa;
            border-radius: 10px;
            border-left: 4px solid #667eea;
        }

        .section-title {
            color: #667eea;
            margin-bottom: 15px;
            font-size: 1.3em;
            font-weight: 600;
        }

        .form-group {
            margin-bottom: 15px;
        }

        label {
            display: block;
            margin-bottom: 5px;
            color: #333;
            font-weight: 500;
        }

        input, textarea, select {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 14px;
            transition: border-color 0.3s;
        }

        input:focus, textarea:focus, select:focus {
            outline: none;
            border-color: #667eea;
        }

        textarea {
            resize: vertical;
            min-height: 100px;
        }

        .btn {
            padding: 12px 24px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 14px;
            font-weight: 600;
            transition: all 0.3s;
        }

        .btn-primary {
            background: #667eea;
            color: white;
        }

        .btn-primary:hover {
            background: #5568d3;
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }

        .btn-secondary {
            background: #6c757d;
            color: white;
            margin-left: 10px;
        }

        .btn-secondary:hover {
            background: #5a6268;
        }

        .btn-success {
            background: #28a745;
            color: white;
            margin-top: 10px;
        }

        .dynamic-item {
            background: white;
            padding: 15px;
            margin-bottom: 15px;
            border-radius: 8px;
            border: 1px solid #e0e0e0;
            position: relative;
        }

        .remove-btn {
            position: absolute;
            top: 10px;
            right: 10px;
            background: #dc3545;
            color: white;
            border: none;
            border-radius: 50%;
            width: 30px;
            height: 30px;
            cursor: pointer;
            font-size: 18px;
            line-height: 1;
        }

        .submit-section {
            text-align: center;
            margin-top: 30px;
            padding-top: 30px;
            border-top: 2px solid #e0e0e0;
        }

        #submit-btn {
            padding: 15px 50px;
            font-size: 18px;
        }

        .loading {
            display: none;
            text-align: center;
            margin-top: 20px;
        }

        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
            margin: 20px auto;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .result {
            display: none;
            margin-top: 20px;
            padding: 20px;
            background: #d4edda;
            border: 1px solid #c3e6cb;
            border-radius: 8px;
            color: #155724;
        }

        .error {
            display: none;
            margin-top: 20px;
            padding: 20px;
            background: #f8d7da;
            border: 1px solid #f5c6cb;
            border-radius: 8px;
            color: #721c24;
        }

        .row {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 15px;
        }

        @media (max-width: 768px) {
            .row {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🚀 AI Resume Generator</h1>
        <p class="subtitle">
            Powered by Llama AI | Create Your Professional Resume
            <span class="new-badge">NEW: LinkedIn & CGPA</span>
        </p>


        <form id="resume-form">
            <!-- Personal Information -->
            <div class="form-section">
                <h2 class="section-title">📋 Personal Information</h2>
                <div class="form-group">
                    <label for="name">Full Name *</label>
                    <input type="text" id="name" name="name" required>
                </div>
                <div class="row">
                    <div class="form-group">
                        <label for="mobile">Mobile Number *</label>
                        <input type="tel" id="mobile" name="mobile" required>
                    </div>
                    <div class="form-group">
                        <label for="email">Email Address *</label>
                        <input type="email" id="email" name="email" required>
                    </div>
                </div>
                <div class="row">
                    <div class="form-group">
                        <label for="location">Current Location *</label>
                        <input type="text" id="location" name="location" required>
                    </div>
                    <div class="form-group">
                        <label for="linkedin">LinkedIn Profile URL <span class="new-badge">NEW</span></label>
                        <input type="url" id="linkedin" name="linkedin" placeholder="https://linkedin.com/in/yourprofile">
                    </div>
                </div>
            </div>

            <!-- Education -->
            <div class="form-section">
                <h2 class="section-title">🎓 Education</h2>
                <div id="qualifications-container">
                    <div class="dynamic-item qualification-item">
                        <div class="row">
                            <div class="form-group">
                                <label>Degree Level *</label>
                                <select class="qualification-degree" required>
                                    <option value="">Select...</option>
                                    <option value="10th">10th Standard</option>
                                    <option value="12th">12th Standard</option>
                                    <option value="Diploma">Diploma</option>
                                    <option value="UG">Undergraduate (UG)</option>
                                    <option value="PG">Postgraduate (PG)</option>
                                    <option value="PhD">PhD</option>
                                </select>
                            </div>
                            <div class="form-group">
                                <label>Specialization</label>
                                <input type="text" class="qualification-spec" placeholder="e.g., Computer Science">
                            </div>
                        </div>
                        <div class="row">
                            <div class="form-group">
                                <label>Institution *</label>
                                <input type="text" class="qualification-institution" required>
                            </div>
                            <div class="form-group">
                                <label>Year of Completion *</label>
                                <input type="text" class="qualification-year" placeholder="e.g., 2023" required>
                            </div>
                            <div class="form-group">
                                <label>CGPA / Percentage *</label>
                                <input type="text" class="qualification-grade" placeholder="8.5 CGPA or 85%" required>
                            </div>
                        </div>
                    </div>
                </div>
                <button type="button" class="btn btn-secondary" onclick="addQualification()">+ Add More Education</button>
            </div>

            <!-- Professional Experience -->
            <div class="form-section">
                <h2 class="section-title">💼 Professional Experience</h2>
                <div class="row">
                    <div class="form-group">
                        <label for="years-experience">Years of Experience *</label>
                        <input type="number" id="years-experience" name="years_experience" min="0" value="0" required>
                    </div>
                    <div class="form-group">
                        <label for="current-role">Current Job Role *</label>
                        <input type="text" id="current-role" name="current_role" placeholder="e.g., Software Engineer or Fresher" required>
                    </div>
                </div>
            </div>

            <!-- Skills -->
            <div class="form-section">
                <h2 class="section-title">🛠️ Skills & Technologies</h2>
                <div class="form-group">
                    <label for="skills">Core Skills * (comma-separated)</label>
                    <input type="text" id="skills" name="skills" placeholder="e.g., Problem Solving, Team Leadership, Communication" required>
                </div>
                <div class="form-group">
                    <label for="tech-stack">Tech Stack * (comma-separated)</label>
                    <input type="text" id="tech-stack" name="tech_stack" placeholder="e.g., Python, React, AWS, Docker" required>
                </div>
            </div>

            <!-- Professional Summary -->
            <div class="form-section">
                <h2 class="section-title">📝 Professional Summary</h2>
                <div class="form-group">
                    <label for="summary">Brief Professional Summary *</label>
                    <textarea id="summary" name="summary" placeholder="Write a brief summary about yourself, your experience, and career goals..." required></textarea>
                </div>
            </div>

            <!-- Projects -->
            <div class="form-section">
                <h2 class="section-title">🚀 Projects</h2>
                <div id="projects-container">
                    <div class="dynamic-item project-item">
                        <div class="form-group">
                            <label>Project Name *</label>
                            <input type="text" class="project-name" required>
                        </div>
                        <div class="form-group">
                            <label>Description *</label>
                            <textarea class="project-description" required></textarea>
                        </div>
                        <div class="form-group">
                            <label>Technologies Used *</label>
                            <input type="text" class="project-technologies" placeholder="e.g., Python, Django, PostgreSQL" required>
                        </div>
                    </div>
                </div>
                <button type="button" class="btn btn-secondary" onclick="addProject()">+ Add More Projects</button>
            </div>

            <!-- Certifications -->
            <div class="form-section">
                <h2 class="section-title">🏆 Certifications (Optional)</h2>
                <div id="certifications-container">
                    <div class="dynamic-item certification-item">
                        <div class="row">
                            <div class="form-group">
                                <label>Certification Name</label>
                                <input type="text" class="cert-name">
                            </div>
                            <div class="form-group">
                                <label>Issuing Organization</label>
                                <input type="text" class="cert-issuer">
                            </div>
                        </div>
                        <div class="form-group">
                            <label>Year</label>
                            <input type="text" class="cert-year" placeholder="e.g., 2023">
                        </div>
                    </div>
                </div>
                <button type="button" class="btn btn-secondary" onclick="addCertification()">+ Add More Certifications</button>
            </div>

            <!-- Submit -->
            <div class="submit-section">
                <button type="submit" id="submit-btn" class="btn btn-primary">🎯 Generate Professional Resume</button>
            </div>
        </form>

        <div class="loading" id="loading">
            <div class="spinner"></div>
            <p>Generating your professional resume with AI enhancement...</p>
        </div>

        <div class="result" id="result"></div>
        <div class="error" id="error"></div>
    </div>

    <script>
        let qualificationCount = 1;
        let projectCount = 1;
        let certificationCount = 1;

        function addQualification() {
            qualificationCount++;
            const container = document.getElementById('qualifications-container');
            const newQual = document.createElement('div');
            newQual.className = 'dynamic-item qualification-item';
            newQual.innerHTML = `
                <button type="button" class="remove-btn" onclick="this.parentElement.remove()">×</button>
                <div class="row">
                    <div class="form-group">
                        <label>Degree Level *</label>
                        <select class="qualification-degree" required>
                            <option value="">Select...</option>
                            <option value="10th">10th Standard</option>
                            <option value="12th">12th Standard</option>
                            <option value="Diploma">Diploma</option>
                            <option value="UG">Undergraduate (UG)</option>
                            <option value="PG">Postgraduate (PG)</option>
                            <option value="PhD">PhD</option>
                        </select>
                    </div>
                    <div class="form-group">
                        <label>Specialization</label>
                        <input type="text" class="qualification-spec" placeholder="e.g., Computer Science">
                    </div>
                </div>
                <div class="row">
                    <div class="form-group">
                        <label>Institution *</label>
                        <input type="text" class="qualification-institution" required>
                    </div>
                    <div class="form-group">
                        <label>Year of Completion *</label>
                        <input type="text" class="qualification-year" placeholder="e.g., 2023" required>
                    </div>
                    <div class="form-group">
                        <label>CGPA / Percentage *</label>
                        <input type="text" class="qualification-grade" placeholder="8.5 CGPA or 85%" required>
                    </div>
                </div>
            `;
            container.appendChild(newQual);
        }

        function addProject() {
            projectCount++;
            const container = document.getElementById('projects-container');
            const newProject = document.createElement('div');
            newProject.className = 'dynamic-item project-item';
            newProject.innerHTML = `
                <button type="button" class="remove-btn" onclick="this.parentElement.remove()">×</button>
                <div class="form-group">
                    <label>Project Name *</label>
                    <input type="text" class="project-name" required>
                </div>
                <div class="form-group">
                    <label>Description *</label>
                    <textarea class="project-description" required></textarea>
                </div>
                <div class="form-group">
                    <label>Technologies Used *</label>
                    <input type="text" class="project-technologies" placeholder="e.g., Python, Django, PostgreSQL" required>
                </div>
            `;
            container.appendChild(newProject);
        }

        function addCertification() {
            certificationCount++;
            const container = document.getElementById('certifications-container');
            const newCert = document.createElement('div');
            newCert.className = 'dynamic-item certification-item';
            newCert.innerHTML = `
                <button type="button" class="remove-btn" onclick="this.parentElement.remove()">×</button>
                <div class="row">
                    <div class="form-group">
                        <label>Certification Name</label>
                        <input type="text" class="cert-name">
                    </div>
                    <div class="form-group">
                        <label>Issuing Organization</label>
                        <input type="text" class="cert-issuer">
                    </div>
                </div>
                <div class="form-group">
                    <label>Year</label>
                    <input type="text" class="cert-year" placeholder="e.g., 2023">
                </div>
            `;
            container.appendChild(newCert);
        }

        document.getElementById('resume-form').addEventListener('submit', async function(e) {
            e.preventDefault();

            // Show loading
            document.getElementById('loading').style.display = 'block';
            document.getElementById('result').style.display = 'none';
            document.getElementById('error').style.display = 'none';
            document.getElementById('submit-btn').disabled = true;

            // Collect form data
            const formData = {
                name: document.getElementById('name').value,
                mobile: document.getElementById('mobile').value,
                email: document.getElementById('email').value,
                location: document.getElementById('location').value,
                linkedin: document.getElementById('linkedin').value,
                years_experience: parseInt(document.getElementById('years-experience').value),
                current_role: document.getElementById('current-role').value,
                skills: document.getElementById('skills').value.split(',').map(s => s.trim()),
                tech_stack: document.getElementById('tech-stack').value.split(',').map(s => s.trim()),
                summary: document.getElementById('summary').value,
                qualifications: [],
                projects: [],
                certifications: []
            };

            // Collect qualifications
            document.querySelectorAll('.qualification-item').forEach(item => {
                const degree = item.querySelector('.qualification-degree').value;
                const spec = item.querySelector('.qualification-spec').value;
                const institution = item.querySelector('.qualification-institution').value;
                const year = item.querySelector('.qualification-year').value;
                const grade = item.querySelector('.qualification-grade').value;

                if (degree && institution && year && grade) {
                    formData.qualifications.push({ degree, specialization: spec, institution, year, grade });
                }
            });

            // Collect projects
            document.querySelectorAll('.project-item').forEach(item => {
                const name = item.querySelector('.project-name').value;
                const description = item.querySelector('.project-description').value;
                const technologies = item.querySelector('.project-technologies').value;

                if (name && description && technologies) {
                    formData.projects.push({ name, description, technologies });
                }
            });

            // Collect certifications
            document.querySelectorAll('.certification-item').forEach(item => {
                const name = item.querySelector('.cert-name').value;
                const issuer = item.querySelector('.cert-issuer').value;
                const year = item.querySelector('.cert-year').value;

                if (name) {
                    formData.certifications.push({ name, issuer, year });
                }
            });

            try {
                const response = await fetch('/generate-resume', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify(formData)
                });

                const result = await response.json();

                if (result.success) {
                    let message = `<h3>✅ Resume Generated Successfully!</h3>`;
                    message += `<p><strong>File:</strong> ${result.filename}</p>`;
                    if (result.s3_url) {
                        message += `<p><strong>S3 URL:</strong> <a href="${result.s3_url}" target="_blank">${result.s3_url}</a></p>`;
                    }
                    message += `<a href="/download/${result.filename}" class="btn btn-success">📥 Download Resume</a>`;

                    document.getElementById('result').innerHTML = message;
                    document.getElementById('result').style.display = 'block';
                } else {
                    throw new Error(result.error || 'Unknown error');
                }
            } catch (error) {
                document.getElementById('error').innerHTML = `<h3>❌ Error</h3><p>${error.message}</p>`;
                document.getElementById('error').style.display = 'block';
            } finally {
                document.getElementById('loading').style.display = 'none';
                document.getElementById('submit-btn').disabled = false;
            }
        });
    </script>
</body>
</html>
    '''

@app.route('/generate-resume', methods=['POST'])
def generate_resume():
    """Generate resume from form data."""
    try:
        user_data = request.json

        # Enhance summary with Llama (optional, can be toggled)
        try:
            enhanced_summary = llama_enhancer.enhance_summary(user_data)
            if enhanced_summary and len(enhanced_summary) > 50:
                user_data['summary'] = enhanced_summary
        except Exception as e:
            print(f"AI enhancement skipped: {e}")

        # Generate PDF
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{user_data['name'].replace(' ', '_')}_{timestamp}.pdf"
        filepath = OUTPUT_DIR / filename

        resume_generator.generate_resume(user_data, str(filepath))

        # Upload to S3 (if configured)
        s3_url = None
        if s3_uploader.s3_client:
            s3_url, error = s3_uploader.upload_file(str(filepath))
            if error:
                print(f"S3 upload failed: {error}")

        return jsonify({
            'success': True,
            'filename': filename,
            's3_url': s3_url,
            'message': 'Resume generated successfully!'
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@app.route('/download/<filename>')
def download_file(filename):
    """Download generated resume."""
    filepath = OUTPUT_DIR / filename
    if filepath.exists():
        return send_file(str(filepath), as_attachment=True)
    return jsonify({'error': 'File not found'}), 404

@app.route('/configure-s3', methods=['POST'])
def configure_s3():
    """Configure AWS S3 credentials."""
    try:
        data = request.json
        s3_uploader.configure_credentials(
            data['aws_access_key'],
            data['aws_secret_key'],
            data['bucket_name'],
            data.get('region', 'us-east-1')
        )
        return jsonify({'success': True, 'message': 'S3 configured successfully'})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

print("✓ Flask app configured")

✓ Flask app configured


## 8. Start the Web Server

In [9]:
# Start ngrok tunnel
from pyngrok import ngrok

# Kill any existing ngrok tunnels
ngrok.kill()
# ngrok.set_auth_token("<provide you ngrok token>")

# Start a new tunnel
public_url = ngrok.connect(5000)
print("\n" + "="*60)
print("🌐 RESUME GENERATOR IS LIVE!")
print("="*60)
print(f"\n📱 Access your app at: {public_url}")
print("\n" + "="*60)
print("\nInstructions:")
print("1. Click the link above to open the form")
print("2. Fill in your details")
print("3. Click 'Generate Professional Resume'")
print("4. Download your AI-enhanced resume!")
print("\n" + "="*60)

# Run Flask app
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("\n✓ Server started! Keep this cell running.\n")
print("Press the stop button in Colab to shut down the server.")


🌐 RESUME GENERATOR IS LIVE!

📱 Access your app at: NgrokTunnel: "https://subcortical-bryon-unafflicted.ngrok-free.dev" -> "http://localhost:5000"


Instructions:
1. Click the link above to open the form
2. Fill in your details
3. Click 'Generate Professional Resume'
4. Download your AI-enhanced resume!


✓ Server started! Keep this cell running.

Press the stop button in Colab to shut down the server.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000


## 9. Create Project ZIP File

In [ ]:
def create_project_zip():
    """Create a downloadable project structure."""

    project_dir = Path('/content/resume_generator_project')
    project_dir.mkdir(exist_ok=True)

    # Create project structure
    (project_dir / 'templates').mkdir(exist_ok=True)
    (project_dir / 'static').mkdir(exist_ok=True)
    (project_dir / 'output').mkdir(exist_ok=True)
    (project_dir / 'models').mkdir(exist_ok=True)

    # Create README
    readme_content = '''# AI Resume Generator

Professional resume generator powered by Llama AI model.

## Features
- Interactive web form
- AI-enhanced content
- Professional PDF generation
- AWS S3 upload support

## Installation

```bash
pip install -r requirements.txt
```

## Usage

### Google Colab (Recommended)
1. Upload `resume_generator_colab.ipynb` to Google Colab
2. Run all cells
3. Access the public URL provided

### Local Execution
```bash
python app.py
```

## Configuration

### AWS S3 (Optional)
Set environment variables:
```bash
export AWS_ACCESS_KEY_ID="your_key"
export AWS_SECRET_ACCESS_KEY="your_secret"
export AWS_BUCKET_NAME="your_bucket"
export AWS_REGION="us-east-1"
```

### Hugging Face Token (Required for some models)
```bash
export HUGGING_FACE_TOKEN="your_token"
```

## Project Structure
```
resume_generator_project/
├── app.py                 # Main Flask application
├── llama_enhancer.py      # AI enhancement module
├── pdf_generator.py       # PDF creation module
├── s3_uploader.py         # AWS S3 integration
├── requirements.txt       # Python dependencies
├── templates/
│   └── index.html        # Frontend interface
├── output/               # Generated resumes
└── README.md
```

## License
MIT License
'''

    with open(project_dir / 'README.md', 'w') as f:
        f.write(readme_content)

    # Create requirements.txt
    requirements = '''transformers>=4.30.0
torch>=2.0.0
accelerate>=0.20.0
reportlab>=4.0.0
boto3>=1.26.0
flask>=2.3.0
flask-cors>=4.0.0
'''

    with open(project_dir / 'requirements.txt', 'w') as f:
        f.write(requirements)

    # Create .gitignore
    gitignore = '''__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
venv/
output/
*.pdf
.env
models/
'''

    with open(project_dir / '.gitignore', 'w') as f:
        f.write(gitignore)

    # Note: The actual Python files are embedded in the notebook
    # For a standalone project, you would extract them

    # Create ZIP file
    zip_path = '/content/resume_generator_project.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(project_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, project_dir.parent)
                zipf.write(file_path, arcname)

    print(f"\n✓ Project ZIP created: {zip_path}")
    print("\nYou can download it from the Colab file browser on the left.")
    return zip_path

# Create the ZIP
zip_file = create_project_zip()

# Provide download link
from google.colab import files
print("\nDownloading project ZIP...")
files.download(zip_file)


✓ Project ZIP created: /content/resume_generator_project.zip

You can download it from the Colab file browser on the left.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 10. Testing & Examples

In [ ]:
# Example: Generate a test resume programmatically

test_data = {
    "name": "John Doe",
    "mobile": "+1-234-567-8900",
    "email": "john.doe@email.com",
    "linkedinurl": "https://www.linkedin.com/in/johndoesasd/",
    "location": "San Francisco, CA",
    "years_experience": 3,
    "current_role": "Software Engineer",
    "skills": ["Problem Solving", "Team Collaboration", "Agile Development"],
    "tech_stack": ["Python", "JavaScript", "React", "Django", "PostgreSQL", "AWS"],
    "summary": "Passionate software engineer with 3 years of experience building scalable web applications. Strong background in full-stack development with expertise in Python and React. Committed to writing clean, maintainable code and delivering high-quality solutions.",
    "qualifications": [
        {
            "degree": "UG",
            "specialization": "Computer Science",
            "institution": "Stanford University",
            "year": "2021",
            "gpa": "3.8"
        },
        {
            "degree": "12th",
            "specialization": "Science",
            "institution": "Lincoln High School",
            "year": "2017",
            "gpa": "3.8"
        }
    ],
    "projects": [
        {
            "name": "E-Commerce Platform",
            "description": "Developed a full-stack e-commerce platform serving 10,000+ users with real-time inventory management and payment processing.",
            "technologies": "React, Node.js, MongoDB, Stripe API"
        },
        {
            "name": "AI Chatbot Assistant",
            "description": "Built an intelligent chatbot using NLP and machine learning to handle customer queries with 85% accuracy.",
            "technologies": "Python, TensorFlow, Flask, Docker"
        }
    ],
    "certifications": [
        {
            "name": "AWS Certified Developer",
            "issuer": "Amazon Web Services",
            "year": "2023"
        }
    ]
}

# Generate test resume
test_filename = "test_resume.pdf"
test_filepath = OUTPUT_DIR / test_filename

resume_generator.generate_resume(test_data, str(test_filepath))
print(f"\n✓ Test resume generated: {test_filepath}")
print("\nYou can download it from the Colab file browser or use the web interface.")

✓ Resume generated: /content/resumes/test_resume.pdf

✓ Test resume generated: /content/resumes/test_resume.pdf

You can download it from the Colab file browser or use the web interface.
